# Sequence Record

Essentially a sequence with database metadata attached to it. Depending on the source, this metadata can be surprisingly rich.

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
record =SeqRecord(Seq("MKQHKAMIVALIVICITAVVAALVTRKD",
 IUPAC.protein),
 id="YP_025292.1", name="HokC",
 description="toxic membrane protein, small")
print record

You get a `SeqRecord` when you:
- parse a file with sequence data in one of the standard formats
- obtain sequence data from an online resource (eg NCBI, UniProt, etc)

# Entrez Utilities - Sequence queries

Online access to NCBI's data. The [Bio.Entrez](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html) module makes use of the Entrez
Programming Utilities (also known as EUtils), described in detail [here at NCBI](http://www.ncbi.nlm.nih.gov/entrez/utils/). You usually get an XML output, which is parsed using some utility function in `Bio.Entrez`.

**Important:** Be aware that the Entrez utilities impose usage limits and you can get yourself blacklisted if you violate those. BioPython takes care of keeping an eye on those for you. You also should fill in a non-fake email address.

# esearch

esearch searches and retrieves primary IDs (for use in EFetch, ELink, and ESummary) and term translations and optionally retains results for future use in the user's environment.


In [ ]:
from Bio import Entrez
Entrez.email = "my_little_self@utu.fi"
handle = Entrez.esearch(db="nucleotide", retmax=10, term="opuntia[ORGN] accD")
print "Handle:", handle #A file-like object, you can read it
content=handle.read()
print "\n\nContent:", content

That's XML allright. Let's parse it. The primary method to do this is `Entrez.read()` but it expects a file. We have already consumed our handle, so it cannot be used again. We could query again, or we can use `StringIO` which is a module which can simulate file objects on top of strings.

In [ ]:
import StringIO
content_filelike=StringIO.StringIO(content)
equery_result=Entrez.read(content_filelike)
print "Parsed result of equery:", equery_result

So we are having a dictionary. Most important here is the `IdList` key which holds the IDs which fit our query. Now we can fetch those.

In [ ]:
handle=Entrez.efetch("nucleotide",id=equery_result['IdList'],retmode="xml")
efetch_result=Entrez.parse(handle)
print "Efetch result", efetch_result #This gives a "generator". You may want to make this into a list, or loop over it
for r in efetch_result:
    print r

This is **not** what we wanted! What we want is a `SeqRecord` object. Reason: `Entrez.parse()` is a generic function for parsing Entrez XML. We are receiving nucleotide database records, ie sequences. For this, we want [Bio.SeqIO](http://biopython.org/DIST/docs/_api_159/Bio.SeqIO-module.html) which is specialized in parsing the various sequence formats we can meet.

In [ ]:
import Bio.SeqIO as SeqIO
handle=Entrez.efetch("nucleotide",id=equery_result['IdList'],rettype="gb")
efetch_result=SeqIO.parse(handle,"gb") #Note: this used to read Entrez.parse(handle)
print "Efetch result", efetch_result #This gives a "generator". You may want to make this into a list, or loop over it
for r in efetch_result:
    print repr(r)
    print
    print

Much better. Now we have SeqRecords. Let us summarize everything into a nice piece of code.

In [ ]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "my_little_self@utu.fi"

def fetch_records(query):
    search_result_handle=Entrez.esearch(db="nucleotide", retmax=10, term=query) #Query Entrez
    search_result=Entrez.read(search_result_handle) #Parse the XML data you get in reply
    efetch_result_handle=Entrez.efetch(db="nucleotide", id=search_result['IdList'],rettype="gb") #Now fetch the actual sequences in GenBank format
    seq_records=SeqIO.parse(efetch_result_handle,format="gb") #Now parse the obtained data using SeqIO
    return seq_records

for seq_rec in fetch_records("opuntia[ORGN] accD"):
    print repr(seq_rec)
    


## Search & download recap (1)

- `Entrez.esearch()` to get a reply with list of IDs
- `Entrez.efetch()` to grab the actual records
- `SeqIO.parse()` to obtain a sequence of SeqRecord objects

This is okay for simple small queries but is highly discouraged for large queries. NCBI expects you to use the *query history* functionality which lets them bind your fetch to a previous search and use their internal caching mechanisms. Turn it on with `usehistory="y"` like such:

In [ ]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "my_little_self@utu.fi"

search_result_handle=Entrez.esearch(db="nucleotide", retmax=10, term="opuntia[ORGN] accD", usehistory="y") #Query Entrez with history on
search_result=Entrez.read(search_result_handle) #Parse the XML data you get in reply
print "Search_result:", search_result


Note the `QueryKey` and `WebEnv` keys. This is the information which identifies your result set at NCBI's server and can be used to fetch the records like so:

In [ ]:
efetch_result_handle=Entrez.efetch(db="nucleotide",
                                   query_key=search_result["QueryKey"],
                                   webenv=search_result["WebEnv"],
                                   rettype="gb") #Now fetch the actual sequences in GenBank format. Note how I don't give the IDs!
seq_records=SeqIO.parse(efetch_result_handle,format="gb") #Now parse the obtained data using SeqIO
for r in seq_records:
    print repr(r)
    print
    print
    

## Search & download recap (2)

- `Entrez.esearch(..., usehistory="y")` to get a reply and webenv + query_key
- `Entrez.efetch(..., webenv=, query_key=)` to grab the actual records from the previous search
- `SeqIO.parse()` to obtain a sequence of SeqRecord objects

Great! Now our problem is that if we have a large result set, thousands of IDs, Entrez doesn't let us grab them all at once with a single efetch. We need to get them bit-by-bit. Here's a complete example.

In [ ]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "my_little_self@utu.fi"

def fetch_records(query, ret_max=10, fetch_batch=5):
    search_result_handle=Entrez.esearch(db="nucleotide", retmax=ret_max, term=query, usehistory="y") #Query Entrez
    search_result=Entrez.read(search_result_handle) #Parse the XML data I get in reply
    #The only thing you care about is Count, and the Webenv+QueryKey data
    all_records=[] #This will be our result list
    #Divide into blocks of fetch_batch results, download one at a time
    for start_index in range(0,len(search_result["IdList"]),fetch_batch):
        efetch_result_handle=Entrez.efetch(db="nucleotide", 
                                           retstart=start_index,
                                           retmax=fetch_batch,
                                           webenv=search_result["WebEnv"],
                                           query_key=search_result["QueryKey"],
                                           rettype="gb") #Now fetch the actual sequences in GenBank format
        #add these records into our list
        all_records.extend(SeqIO.parse(efetch_result_handle,format="gb")) #Now parse the obtained data using SeqIO
    return all_records

fetched_records_p53=fetch_records("p53", ret_max=50, fetch_batch=10) #get max 50 records, fetch 10 at a time
print "Fetched", len(fetched_records_p53), "records"
print "First few:", fetched_records_p53[:5]
